In [1]:
# Imports 
# Env var
import os 
import sys
import markdownify

from dotenv import load_dotenv, find_dotenv

In [2]:
# Env variable
sys.path.append('../')
load_dotenv(find_dotenv())

True

# 1. Confluence Loader 

In [3]:
from config import (CONFLUENCE_SPACE_NAME, CONFLUENCE_SPACE_KEY,
                    CONFLUENCE_USERNAME, CONFLUENCE_API_KEY, PERSIST_DIRECTORY)

In [4]:
from langchain.document_loaders import ConfluenceLoader
loader = ConfluenceLoader(
    url=CONFLUENCE_SPACE_NAME,
    username=CONFLUENCE_USERNAME,
    api_key=CONFLUENCE_API_KEY
)

In [5]:
docs = loader.load(
    space_key=CONFLUENCE_SPACE_KEY,
    limit=10,
    # include_attachments=True, # uncomment to include png, jpeg, ..
    max_pages=50,
    keep_markdown_format=True
)

In [6]:
# Look at one page content and its metadata
print("Content: \n ------- \n" + docs[-1].page_content)
print("Metadatas: \n ------- \n" + str(docs[-1].metadata))

Content: 
 ------- 
## Qu'est-ce qu'un Comité d'Entreprise ?

Le Comité d'Entreprise (CE) est une institution représentative du personnel présente dans les entreprises françaises. Il a été créé pour assurer la représentation des salariés et leur permettre de participer activement aux décisions concernant leur vie au sein de l'entreprise. Le Comité d'Entreprise est obligatoire dans les entreprises de certaines tailles et ses missions sont définies par le Code du Travail.

## Comment accéder à mon CE ?

Pour accéder au CE, vous pouvez consultant l’adresse suivante: <https://mon-CE.fr>

Si vous n’avez pas vos identifiants, vous pouvez envoyer un mail à [xxxx@mon-ce.fr](mailto:xxxx@mon-ce.fr), le responsable du comité d’entreprise. 

Renseignez les identifiants sur le site du CE et vous pourrez bénéficier de nombreux avantages.

## Rôle du Comité d'Entreprise :

Le Comité d'Entreprise joue un rôle essentiel dans le dialogue social au sein de l'entreprise. Ses principales missions sont les 

In [7]:
def pretty_print(chunks):
    print(
        str('\n' + '='*50 + '\n').join(
            [
                chunk.page_content + '\n' +'-'*50 + '\n' + str(chunk.metadata) 
                for chunk in chunks
            ]
        )
    )

## 2. Document Splitter 

### Document Example

In [8]:
import langchain
text = """
# Je suis un titre 
## Je suis un sous-titre

Je suis un bloc de texte. Cependant, ma taille est assez longue. J'aimerais dans un premier temps que le MarkdownHeaderTextSplitter
identifie mon titre et sous-titre dans ses métadonnées.

Je souhaite ensuite que RecursiveCharacterTextSplitter identifie les deux parties qui me composent 
car ma taille serait trop volumineuse pour alimenter un modèle de langue. 

Enfin j'aimerais que les métadatas correspondant à mes origines, à savoir l'url, soit mergées avec mes informations
de titre et sous titres.
"""

metadata={'url': 'https://mon_origine.com'}

sample = langchain.schema.document.Document(page_content=text, metadata=metadata)

### MarkdownHeaderTextSplitter example

In [9]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# Markdown 
headers_to_split_on = [
    ("#", "Titre 1"),
    ("##", "Sous-titre 1"),
    ("###", "Sous-titre 2"),
]

# Markdown splitter
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
chunks = markdown_splitter.split_text(sample.page_content)

print(chunks)

[Document(page_content="Je suis un bloc de texte. Cependant, ma taille est assez longue. J'aimerais dans un premier temps que le MarkdownHeaderTextSplitter\nidentifie mon titre et sous-titre dans ses métadonnées.  \nJe souhaite ensuite que RecursiveCharacterTextSplitter identifie les deux parties qui me composent\ncar ma taille serait trop volumineuse pour alimenter un modèle de langue.  \nEnfin j'aimerais que les métadatas correspondant à mes origines, à savoir l'url, soit mergées avec mes informations\nde titre et sous titres.", metadata={'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre'})]


### RecursiveCharacterTextSplitter example

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=20,
    separators=["#", "\n\n", "\n", "(?<=\. )", " ", ""]
)

splitted_chunks = splitter.split_documents(chunks)

pretty_print(splitted_chunks)

Je suis un bloc de texte. Cependant, ma taille est assez longue. J'aimerais dans un premier temps que le MarkdownHeaderTextSplitter
identifie mon titre et sous-titre dans ses métadonnées.
--------------------------------------------------
{'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre'}
Je souhaite ensuite que RecursiveCharacterTextSplitter identifie les deux parties qui me composent
car ma taille serait trop volumineuse pour alimenter un modèle de langue.
--------------------------------------------------
{'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre'}
Enfin j'aimerais que les métadatas correspondant à mes origines, à savoir l'url, soit mergées avec mes informations
de titre et sous titres.
--------------------------------------------------
{'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre'}


### MarkdownHeaderTextSplitter & RecursiveCharacterTextSplitter

In [11]:
# Markdown 
headers_to_split_on = [
    ("#", "Titre 1"),
    ("##", "Sous-titre 1"),
    ("###", "Sous-titre 2"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Split based on markdown and add original metadata
md_docs = []
for doc in [sample]:
    md_doc = markdown_splitter.split_text(doc.page_content)
    for i in range(len(md_doc)):
        md_doc[i].metadata = md_doc[i].metadata | doc.metadata 
    md_docs.extend(md_doc)

# RecursiveTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Chunk size big enough
splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

splitted_docs = splitter.split_documents(md_docs)

pretty_print(splitted_docs)

Je suis un bloc de texte. Cependant, ma taille est assez longue. J'aimerais dans un premier temps que le MarkdownHeaderTextSplitter
identifie mon titre et sous-titre dans ses métadonnées.
--------------------------------------------------
{'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre', 'url': 'https://mon_origine.com'}
Je souhaite ensuite que RecursiveCharacterTextSplitter identifie les deux parties qui me composent
car ma taille serait trop volumineuse pour alimenter un modèle de langue.
--------------------------------------------------
{'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre', 'url': 'https://mon_origine.com'}
Enfin j'aimerais que les métadatas correspondant à mes origines, à savoir l'url, soit mergées avec mes informations
de titre et sous titres.
--------------------------------------------------
{'Titre 1': 'Je suis un titre', 'Sous-titre 1': 'Je suis un sous-titre', 'url': 'https://mon_origine.com'}


In [12]:
def my_custom_splitter(docs):
    # Markdown 
    headers_to_split_on = [
        ("#", "Titre 1"),
        ("##", "Sous-titre 1"),
        ("###", "Sous-titre 2"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    
    # Split based on markdown and add original metadata
    md_docs = []
    for doc in docs:
        md_doc = markdown_splitter.split_text(doc.page_content)
        for i in range(len(md_doc)):
            md_doc[i].metadata = md_doc[i].metadata | doc.metadata 
        md_docs.extend(md_doc)
    
    # RecursiveTextSplitter
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    # Chunk size big enough
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        separators=["\n\n", "\n", "(?<=\. )", " ", ""]
    )
    
    splitted_docs = splitter.split_documents(md_docs)
    return splitted_docs

In [13]:
chunks = my_custom_splitter(docs)

# 3. Embeddings & Vector DB 

In [14]:
persist_directory = './db/chroma'

In [15]:
# Embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [16]:
import shutil
try:
    shutil.rmtree(persist_directory)
except FileNotFoundError as e:
    pass

In [17]:
# Save db 
from langchain.vectorstores import Chroma
db = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)
db.persist()

In [18]:
# Count the number of chunks in the vector store
db._collection.count()

112

In [19]:
# db.get()
retriever = db.as_retriever()
# retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.3})

In [20]:
from langchain.prompts import PromptTemplate
template = """Given this text extracts:
    -----
    {context}
    -----
    Please answer with to the following question:
    Question: {question}
    Answer: 
    """

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [21]:
# LLM
from langchain.llms import OpenAI  
llm = OpenAI(streaming=True)

In [22]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": prompt}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # or 
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)

In [23]:
question = "Comment accéder à mon comité d'entreprise ? "

query = {"query": question}
answer = qa(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [24]:
from IPython.display import display_markdown
display_markdown(answer["result"], raw=True)

 Vous pouvez accéder à votre Comité d'Entreprise en consultant l'adresse suivante: <https://mon-CE.fr>. Si vous n'avez pas vos identifiants, vous pouvez envoyer un mail à [xxxx@mon-ce.fr](mailto:xxxx@mon-ce.fr). Renseignez les identifiants sur le site du CE et vous pourrez bénéficier de nombreux avantages.

### Display documents used by the LLM for answering

In [25]:
retriever.get_relevant_documents("Comment accéder à mon comité d'entreprise ?")

[Document(page_content='Pour accéder au CE, vous pouvez consultant l’adresse suivante: <https://mon-CE.fr>  \nSi vous n’avez pas vos identifiants, vous pouvez envoyer un mail à [xxxx@mon-ce.fr](mailto:xxxx@mon-ce.fr), le responsable du comité d’entreprise.  \nRenseignez les identifiants sur le site du CE et vous pourrez bénéficier de nombreux avantages.', metadata={'Sous-titre 1': 'Comment accéder à mon CE ?', 'title': "Comité d'entreprise (CE) - Définition et rôles", 'id': '1933313', 'source': 'https://florianbastin.atlassian.net/wiki/spaces/~70121e1c1cf2b203a49dabc6762c43bdbfe05/pages/1933313'}),
 Document(page_content="Le Comité d'Entreprise est chargé de veiller au respect des droits des salariés en matière de travail, de sécurité, de santé et de conditions de travail. Il peut être consulté par la direction de l'entreprise sur différentes questions liées à l'organisation du travail, aux licenciements collectifs, aux restructurations, etc. Il est également informé des projets de l'e